<a href="https://colab.research.google.com/github/DayrisRM/VisionArtificial-M1/blob/main/ROI_GT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data.zip to data.zip


In [ ]:
import zipfile
import io
data = zipfile.ZipFile(io.BytesIO(uploaded['data.zip']), 'r')
data.extractall()

In [ ]:
data.printdir()

In [1]:
import cv2
import numpy as np
import os

import matplotlib.pyplot as plt
import plot_cv_utils

In [ ]:
from IPython.core.display import Image

def createRoiFromImage(image):  
  im = cv2.imread(image)
  imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  ret, thresh = cv2.threshold(imgray, 150, 255, 0)
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

  img_contours2 = cv2.drawContours(im.copy(), contours,
                                 -1, (0, 255, 0), 3)

  plot_cv_utils.subplot_cv_list([im, img_contours2], 
                              ['Original', 'Contornos'], False, False)

  # Número total de objetos 
  print("Objetos {}".format(len(contours)))
  print(ret)


------- Step 1: Find and crop ROI images ------

In [ ]:
def findAndCropROI(imageName):  
  try:
    imagePath = "/content/GT/{}".format(imageName)
    imageUSPath = "/content/Original/{}".format(imageName)
    croppedRoiPath = '/content/drive/MyDrive/Colab Notebooks/US_ROI/'

    im = cv2.imread(imagePath)
    imUS = cv2.imread(imageUSPath)

    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    cnts = cv2.findContours(imgray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    ROI_number = 0
    margin = 8
    for c in cnts:
      x,y,w,h = cv2.boundingRect(c)
      ROI = imUS[y-margin:y+h+margin, x-margin:x+w+margin]
      roiPath = croppedRoiPath + 'ROI_{}{}'.format(ROI_number, imageName)      
      cv2.imwrite(roiPath, ROI)
      ROI_number += 1
  except:
    print("An exception occurred_{}".format(imageName)) 



In [ ]:

myListImages = os.listdir('/content/GT')
for cl in myListImages: 
  print(cl)
  findAndCropROI(cl)
  


------- Step 2:  Preparing dataset from ROI images   ------

In [22]:
import cv2
import numpy as np


In [23]:
#obtener el width y height maximo de las imagenes a preparar
def getMaxSize(folderUrl):
  moreHeight = 0
  moreWidht = 0
  myListImages = os.listdir(folderUrl)
  for cl in myListImages: 
    img = cv2.imread(folderUrl + cl)
    old_image_height, old_image_width, channels = img.shape

    if old_image_height > moreHeight:
      moreHeight = old_image_height

    if old_image_width > moreWidht:
      moreWidht = old_image_width

  return moreHeight, moreWidht

In [30]:
#añadir padding a la imagen original con las nuevas medidas
def addPaddingToImage(imageName, new_height, new_width): 
  # read image  
  img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/US_ROI/' + imageName)
  old_image_height, old_image_width, channels = img.shape 
  color = (0,0,0)
  result = np.full((new_height,new_width, channels), color, dtype=np.uint8)

  # compute center offset
  x_center = (new_width - old_image_width) // 2
  y_center = (new_height - old_image_height) // 2

  # copy img image into center of result image
  result[y_center:y_center+old_image_height, 
  x_center:x_center+old_image_width] = img

  print(imageName)
  # save result
  urlToSave = '/content/drive/MyDrive/Colab Notebooks/US_ROI_Prepared/Padding/' + imageName  
  cv2.imwrite(urlToSave, result)
  

In [35]:
from PIL import Image

#resize la imagen original con las nuevas medidas 
def resizeImage(imageName): 
  basewidth = 224
  img = Image.open('/content/drive/MyDrive/Colab Notebooks/US_ROI_Prepared/Padding/' + imageName )
  wpercent = (basewidth / float(img.size[0]))
  hsize = int((float(img.size[1]) * float(wpercent)))
  img = img.resize((basewidth, hsize), Image.ANTIALIAS)
  img.save('/content/drive/MyDrive/Colab Notebooks/US_ROI_Prepared/ResizedImage/{}'.format(imageName))
  



In [36]:
def prepareROIImages():
  new_height = 0
  new_width = 0  

  moreHeight, moreWidht = getMaxSize('/content/drive/MyDrive/Colab Notebooks/US_ROI/')

  if moreHeight < 500 and moreWidht < 500:
    new_height = 500
    new_width = 500
  
  myListImages = os.listdir('/content/drive/MyDrive/Colab Notebooks/US_ROI/')
  for cl in myListImages: 
    addPaddingToImage(cl, new_height, new_width)
    resizeImage(cl)


In [ ]:
prepareROIImages()